<a href="https://colab.research.google.com/github/caro28/stinky/blob/master/stinky_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary

*   This file loads and merges data from two databases for residents to report odor complaints: [SmellMyCity (SMC)](https://smellmycity.org/) and [SeeClickFix (SCF)](https://seeclickfix.com/portland_2). The data is from 2020 and 2021.
* Running all cells (Runtime --> Run all) will automatically download df_stinky.csv, on the user's computer, after loading and merging the raw data.
* The majority of SMC data is from S Portland, though it does contain data from Portland as well. SCF contained only Portland data when this notebook was created (Aug 2021).
* New columns were added to df_stinky to filter reports by city (South Portland/Portland) and to look at date/time more granularly.

## How to update df_stinky (table that combines data from SMC and SCF data)
1. Update raw SMC data and SCF data: 
   * SMC data: download from [SMC website](https://smellmycity.org/data). Select by zipcode, enter "4101,4102,4106,4107,4103,4108,4124", download and save as csv (use this file name: smc.csv)
   * Get updated SCF Excel spreadsheet from city of Portland and save as csv (use this file name: scf.csv)
   *   Upload updated csv files from SmellMyCity (SMC) and SeeClickFix (SCF) in [data folder](https://github.com/ds5110/stinky/tree/master/data)
2. Run all cells --> this will download a csv file containing df_stinky
3. Upload df_stinky.csv to [data folder](https://github.com/ds5110/stinky/tree/master/data). (Do not need to delete old df_stinky.csv file; GitHub will automatically replace with new file.)

Note: if updated data for SCF is sent through new, separate Excel spreadsheets, the code for loading data will need to be edited (e.g. how raw data for oil vessel arrivals is loaded in [oil_vessels_data.ipynb](https://github.com/ds5110/stinky/blob/master/oil_vessel_dataset.ipynb))

# Portland and S Portland zip codes included in current SMC and SCF raw data

*   Portland
  *   04101, 04102: Portland immediately north of Fore River
  *   04103: Portland, further north (to northern border of the city)
  *   04108: Portland, islands to the east (Peaks Island and Cushing Island)
  *   04124: Portland, west of airport 

*   South Portland
  *   04106, 04107: S Portland immediately south of Fore River


In [1]:
import pandas as pd

# load SMC data
url_all_zips = 'https://raw.githubusercontent.com/ds5110/stinky/master/data/smc.csv'
df_smc = pd.read_csv(url_all_zips)
print(df_smc.shape)

# the first rows of df_smc look like this
df_smc.head()

(2612, 9)


,epoch time,date & time,smell value,skewed latitude,skewed longitude,zipcode,smell description,symptoms,additional comments
0,1558531873,05/22/2019 09:31:13 -04:00,3,43.6608,-70.2498,4101,Grainy / malt like - coming from St. John street,NaN,NaN
1,1558691615,05/24/2019 05:53:35 -04:00,3,43.6435,-70.2702,4102,NaN,NaN,NaN
2,1559178135,05/29/2019 21:02:15 -04:00,3,43.6466,-70.2770,4102,Asphalty,NaN,NaN
3,1559341934,05/31/2019 18:32:14 -04:00,1,43.6325,-70.2828,4106,NaN,NaN,NaN
4,1559387558,06/01/2019 07:12:38 -04:00,3,43.6343,-70.2825,4106,Oil fumes,Throat irritation,NaN


In [2]:
# Check for NA values
df_smc.isna().sum()

epoch time                0
date & time               0
smell value               0
skewed latitude           0
skewed longitude          0
zipcode                   0
smell description       560
symptoms               1678
additional comments    2549
dtype: int64

In [3]:
# get number of reports by zipcode (look at values in columns with 0 NA values)
df_smc.groupby('zipcode').count()

,epoch time,date & time,smell value,skewed latitude,skewed longitude,smell description,symptoms,additional comments
zipcode,,,,,,,,
4101,53,53,53,53,53,43,16,2
4102,376,376,376,376,376,254,122,11
4103,5,5,5,5,5,4,4,1
4106,2169,2169,2169,2169,2169,1748,791,49
4107,7,7,7,7,7,3,1,0
4108,1,1,1,1,1,0,0,0
4124,1,1,1,1,1,0,0,0


In [4]:
# add column for city name
df_smc['South Portland/Portland'] = 'Portland'
df_smc.loc[df_smc['zipcode'].isin([4106, 4107]), 'South Portland/Portland'] = "South Portland"

In [5]:
# find out number of Portland vs S Portland rows
df_smc.groupby('South Portland/Portland').count()

,epoch time,date & time,smell value,skewed latitude,skewed longitude,zipcode,smell description,symptoms,additional comments
South Portland/Portland,,,,,,,,,
Portland,436,436,436,436,436,436,301,142,14
South Portland,2176,2176,2176,2176,2176,2176,1751,792,49


In [6]:
def separate_datecols(df, datetime_col):
  # create separate columns for day, month, year, hour and month name
  df['Day'] = col.dt.day
  df['Month'] = col.dt.month
  df['Year'] = col.dt.year
  df['Hour'] = col.dt.hour
  df['Month_name'] = pd.to_datetime(df['Month'], format='%m').dt.month_name().str.slice(stop=3)

  # Create a date and hour column
  df['Date & time (hour rounded)'] = col.dt.strftime("%Y-%m-%d %H:00:00")

In [7]:
# change the date and time column to datetime format
df_smc['date & time'] = df_smc['date & time'].str[0:20]
df_smc['date & time'] = pd.to_datetime(df_smc['date & time'])

# create separate columns for date and time
df_smc['date'] = [d.date() for d in df_smc['date & time']]
df_smc['time'] = [d.time() for d in df_smc['date & time']]

# create separate columns for day, month, year, hour and month name
col = df_smc['date & time']
separate_datecols(df_smc, col)

# rename columns
df_smc.rename(columns={'skewed latitude':'Latitude', 'skewed longitude':'Longitude'}, inplace=True)

In [8]:
# load SCF data
url = 'https://raw.githubusercontent.com/ds5110/stinky/master/data/scf.csv'
df_scf=pd.read_csv(url)
print(df_scf.shape)

# the first rows of df_scf look like this
df_scf.head()

(295, 12)


,Id,Report Source,Category,Created at local,Closed at local,Status,Address,Description,URL,Lat,Lng,Export tagged places
0,7181157,iPhone,Odor,01/07/2020 - 08:26AM,01/07/2020 - 09:20AM,Archived,315 Spring Street,Petroleum smell coming from south portland,https://crm.seeclickfix.com/#/organizations/61...,43.647740,-70.269455,City Council District 2
1,7181402,Android,Odor,01/07/2020 - 09:11AM,01/07/2020 - 09:20AM,Archived,25 Cushman St,usual petroleum,https://crm.seeclickfix.com/#/organizations/61...,43.649448,-70.268626,City Council District 2
2,7192000,Android,Odor,01/09/2020 - 07:14AM,01/09/2020 - 08:45AM,Archived,25 Cushman St,usual petroleum,https://crm.seeclickfix.com/#/organizations/61...,43.649448,-70.268626,City Council District 2
3,7206428,Android,Odor,01/13/2020 - 08:22AM,01/13/2020 - 09:09AM,Archived,25 Cushman St,worst yet,https://crm.seeclickfix.com/#/organizations/61...,43.649448,-70.268626,City Council District 2
4,7210067,Android,Odor,01/14/2020 - 08:24AM,01/14/2020 - 02:50PM,Archived,25 Cushman St,usual petroleum stink. Cushman and Reiche play...,https://crm.seeclickfix.com/#/organizations/61...,43.649448,-70.268626,City Council District 2


In [9]:
# check for NA values 
df_scf.isnull().sum()

Id                       0
Report Source            0
Category                 0
Created at local         0
Closed at local          2
Status                   0
Address                  0
Description             39
URL                      0
Lat                      0
Lng                      0
Export tagged places     0
dtype: int64

In [10]:
# change the date and time column to datetime format
df_scf['Created at local']=pd.to_datetime(df_scf['Created at local'])
df_scf['Closed at local']=pd.to_datetime(df_scf['Closed at local'])

# create separate columns for date and time
df_scf['date'] = [d.date() for d in df_scf['Created at local']]
df_scf['time'] = [d.time() for d in df_scf['Created at local']]

# create separate columns for day, month, year, hour and month name
col = df_scf['Created at local']
separate_datecols(df_scf, col)

# rename columns
df_scf.rename(columns={'Description':'smell description', 'Lat':'Latitude', 'Lng':'Longitude'}, inplace=True)

In [11]:
# we used Google's geocoding API to determine the city of odor reports in df_scf, which contains only addresses (number and street name) and lat/long
# This is in case future SCF data includes reports from South Portland.

# install packages for google geocoding API
!apt install gdal-bin python-gdal python3-gdal --quiet
!apt install python3-rtree --quiet
!pip install git+git://github.com/geopandas/geopandas.git --quiet
!pip install descartes --quiet
!pip install geopy
!pip install plotly_express
!pip install ipython-autotime
#!pip install tqdm==4.36.0
!pip install tqdm==4.62
%load_ext autotime
import pandas as pd

import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

import matplotlib.pyplot as plt
import plotly_express as px
import tqdm
from tqdm import tqdm
from tqdm._tqdm_notebook import tqdm_notebook

Reading package lists...
Building dependency tree...
Reading state information...
gdal-bin is already the newest version (2.2.3+dfsg-2).
python-gdal is already the newest version (2.2.3+dfsg-2).
python3-gdal is already the newest version (2.2.3+dfsg-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.
Reading package lists...
Building dependency tree...
Reading state information...
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libspatialindex-c4v5 libspatialindex-dev libspatialindex4v5
  python3-pkg-resources
Suggested packages:
  python3-setuptools
The following NEW packages will be installed:
  libspatialindex-c4v5 libspatialindex-dev libspatialindex4v5
  python3-pkg-resources python3-rtree
0 up

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`



In [12]:
# THIS CELL TAKES ~2.5 minutes to run (with 295 rows of data)

# create column "address" from lat and long
df_scf["geom"] =  df_scf["Latitude"].map(str)  + ',' + df_scf['Longitude'].map(str)
locator = Nominatim(user_agent="myGeocoder", timeout=10)
rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.001)
tqdm.pandas()
df_scf['address'] = df_scf['geom'].progress_apply(rgeocode)

100%|██████████| 295/295 [02:27<00:00,  2.00it/s]

time: 2min 27s (started: 2021-08-21 13:46:43 +00:00)


In [13]:
# create 'South Portland/Portland' column and filter out any South Portland entries
import re
df_scf['South Portland/Portland'] = 'Portland'
df_scf.loc[df_scf['address'].str.contains('South Portland', na=False), 'South Portland/Portland'] = "South Portland"

time: 10.1 ms (started: 2021-08-21 13:49:10 +00:00)


In [14]:
# Merge two datasets
df_stinky = df_scf.append(df_smc, sort=False)

# the first rows of df_stinky look like this
df_stinky.head()

,Id,Report Source,Category,Created at local,Closed at local,Status,Address,smell description,URL,Latitude,Longitude,Export tagged places,date,time,Day,Month,Year,Hour,Month_name,Date & time (hour rounded),geom,address,South Portland/Portland,epoch time,date & time,smell value,zipcode,symptoms,additional comments
0,7181157.0,iPhone,Odor,2020-01-07 08:26:00,2020-01-07 09:20:00,Archived,315 Spring Street,Petroleum smell coming from south portland,https://crm.seeclickfix.com/#/organizations/61...,43.647740,-70.269455,City Council District 2,2020-01-07,08:26:00,7,1,2020,8,Jan,2020-01-07 08:00:00,"43.64774,-70.26945450000001","(315, Spring Street, West End, Portland, Cumbe...",Portland,NaN,NaT,NaN,NaN,NaN,NaN
1,7181402.0,Android,Odor,2020-01-07 09:11:00,2020-01-07 09:20:00,Archived,25 Cushman St,usual petroleum,https://crm.seeclickfix.com/#/organizations/61...,43.649448,-70.268626,City Council District 2,2020-01-07,09:11:00,7,1,2020,9,Jan,2020-01-07 09:00:00,"43.6494485,-70.2686259","(22, Cushman Street, West End, Portland, Cumbe...",Portland,NaN,NaT,NaN,NaN,NaN,NaN
2,7192000.0,Android,Odor,2020-01-09 07:14:00,2020-01-09 08:45:00,Archived,25 Cushman St,usual petroleum,https://crm.seeclickfix.com/#/organizations/61...,43.649448,-70.268626,City Council District 2,2020-01-09,07:14:00,9,1,2020,7,Jan,2020-01-09 07:00:00,"43.6494485,-70.2686259","(22, Cushman Street, West End, Portland, Cumbe...",Portland,NaN,NaT,NaN,NaN,NaN,NaN
3,7206428.0,Android,Odor,2020-01-13 08:22:00,2020-01-13 09:09:00,Archived,25 Cushman St,worst yet,https://crm.seeclickfix.com/#/organizations/61...,43.649448,-70.268626,City Council District 2,2020-01-13,08:22:00,13,1,2020,8,Jan,2020-01-13 08:00:00,"43.6494485,-70.2686259","(22, Cushman Street, West End, Portland, Cumbe...",Portland,NaN,NaT,NaN,NaN,NaN,NaN
4,7210067.0,Android,Odor,2020-01-14 08:24:00,2020-01-14 14:50:00,Archived,25 Cushman St,usual petroleum stink. Cushman and Reiche play...,https://crm.seeclickfix.com/#/organizations/61...,43.649448,-70.268626,City Council District 2,2020-01-14,08:24:00,14,1,2020,8,Jan,2020-01-14 08:00:00,"43.6494485,-70.2686259","(22, Cushman Street, West End, Portland, Cumbe...",Portland,NaN,NaT,NaN,NaN,NaN,NaN


time: 68.7 ms (started: 2021-08-21 13:49:10 +00:00)


In [15]:
# check that common columns were correctly merged
df_stinky.isnull().sum()

Id                            2612
Report Source                 2612
Category                      2612
Created at local              2612
Closed at local               2614
Status                        2612
Address                       2612
smell description              599
URL                           2612
Latitude                         0
Longitude                        0
Export tagged places          2612
date                             0
time                             0
Day                              0
Month                            0
Year                             0
Hour                             0
Month_name                       0
Date & time (hour rounded)       0
geom                          2612
address                       2612
South Portland/Portland          0
epoch time                     295
date & time                    295
smell value                    295
zipcode                        295
symptoms                      1973
additional comments 

time: 13.9 ms (started: 2021-08-21 13:49:10 +00:00)


In [16]:
# look into missing 'smell description' data to check there were no merge errors
print('SCF has {} null smell description rows'.format(df_scf['smell description'].isnull().sum()))
print('SMC has {} null smell description rows'.format(df_smc['smell description'].isnull().sum()))
print('Merged df has {} null smell description rows'.format(df_stinky['smell description'].isnull().sum()))

SCF has 39 null smell description rows
SMC has 560 null smell description rows
Merged df has 599 null smell description rows
time: 10.3 ms (started: 2021-08-21 13:49:10 +00:00)


In [17]:
# download tidied df_stinky
from google.colab import files
df_stinky.to_csv('df_stinky.csv', index=False) 
files.download('df_stinky.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

time: 68.5 ms (started: 2021-08-21 13:49:10 +00:00)
